In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dmatrix_df = pd.read_csv("distance_matrix.csv")
dmatrix_df.head()

FileNotFoundError: [Errno 2] File b'distance_matrix.csv' does not exist: b'distance_matrix.csv'

In [ ]:
dmatrix_df.drop(dmatrix_df.columns[0], axis =1, inplace = True)
dmatrix_df.head()

In [ ]:
dmatrix_df = dmatrix_df[['mode', 'origin', 'destination', 'distance', 'duration']]
dmatrix_df.head()

In [ ]:
dmatrix_mode_pivot = pd.pivot_table(dmatrix_df, values = 'duration', index = ['origin', 'destination'], columns = 'mode')
dmatrix_mode_pivot

In [ ]:
for group_name, grp in dmatrix_df.groupby('origin'):
    plot_table = grp.pivot(index='destination', columns='mode', values='duration')
    plot_table.plot(kind = 'bar', title = group_name, figsize = (15,7))

In [ ]:
dmatrix_mode_pivot = pd.DataFrame(dmatrix_mode_pivot.to_records())
dmatrix_mode_pivot.head()

In [ ]:
dmatrix_mode_pivot = dmatrix_mode_pivot.drop(dmatrix_mode_pivot[dmatrix_mode_pivot['origin'] == dmatrix_mode_pivot['destination']].index, axis=0).reset_index(drop = True)
dmatrix_mode_pivot.head()

In [ ]:
dm_driving = dmatrix_mode_pivot[['origin', 'destination','driving']].copy()
dm_driving

In [ ]:
average_time =[]
for index, row in dm_driving.iterrows():
    origin = row['origin']
    destination = row['destination']
    for index1, row1 in dm_driving.iterrows():
        if(row1['origin'] == destination and row1['destination'] == origin):
            average = (row['driving'] + row1['driving']) / 2
            average_time.append(average)
average_time

In [ ]:
dm_driving['average_drive_time'] = average_time
dm_driving

In [ ]:
origin_zipcodes = dm_driving['origin'].unique()
origin_zipcodes

In [ ]:
nearest_facility = []
for origin in origin_zipcodes:
    origin_df = dm_driving[dm_driving['origin'] == origin].copy()
    print(origin_df)
    print(origin_df.average_drive_time.idxmin())
    d = {'origin': origin, 'destination': origin_df['destination'][origin_df['average_drive_time'].idxmin()], 'average_time': origin_df['average_drive_time'].min(0)}
    nearest_facility.append(d)
nearest_facility

In [ ]:
near_area_zipcodes_df = pd.DataFrame(nearest_facility)
near_area_zipcodes_df

In [ ]:
near_area_zipcodes_df = near_area_zipcodes_df[['origin','destination','average_time']]
near_area_zipcodes_df

In [ ]:
facilities_df = pd.read_csv("facilities.csv")
facilities_df

In [ ]:
facilities_df = (facilities_df.merge(near_area_zipcodes_df, left_on='Facility Area-Zipcode', right_on='origin').reindex(columns=['Facility ID', 'Facility Area-Zipcode', 'destination', 'average_time','Facility Staff Count']))
facilities_df

In [ ]:
population_df = pd.read_csv("facility_pop_2017/ACS_17_5YR_DP05_with_ann.csv")
population_df

In [ ]:
population_df.columns = population_df.iloc[0]
population_df = population_df[1:].reset_index()
population_df

In [ ]:
population_df['Id2'] = population_df['Id2'].astype(int)
population_df['Estimate; SEX AND AGE - Total population'] = population_df['Estimate; SEX AND AGE - Total population'].astype(int)
population_df = population_df.rename(columns = {'Estimate; SEX AND AGE - Total population' : 'Population'})

In [ ]:
population_df = population_df[population_df.columns[0:5]]
population_df

In [ ]:
facilities_df = (facilities_df.merge(population_df, left_on='Facility Area-Zipcode', right_on='Id2').reindex(columns=['Facility ID', 'Facility Area-Zipcode', 'destination', 'average_time', 'Facility Staff Count', 'Population']))
facilities_df

In [ ]:
facilities_df.drop('average_time', axis = 1, inplace = True)
facilities_df = facilities_df.rename({'Facility ID' : 'FacilityID', 'Facility Area-Zipcode' : 'AreaZipcode', 'destination' : 'NearestAreaZipcode', 'Facility Staff Count' : 'StaffCount'}, axis = 'columns')
facilities_df

In [ ]:
facilities_df['Ratio'] = facilities_df['StaffCount']/(facilities_df['Population'])
facilities_df

In [ ]:
def pred_staff(population, ratio):
    y = (population) * ratio
    return y

In [ ]:
y_pred = pred_staff(facilities_df['Population'], facilities_df['Ratio'])
y_pred

In [ ]:
min_ratio = 1/2808

In [ ]:
y_pred_ = pred_staff(facilities_df['Population'], min_ratio)

In [ ]:
y_pred_

In [ ]:
import math
y_pred_ = y_pred_.apply(lambda x: math.ceil(x))
y_pred_

In [ ]:
min_staff_in_area_facility_df = pd.DataFrame({'ZipCode': facilities_df['AreaZipcode'], 'MinStaff': y_pred_})
min_staff_in_area_facility_df

In [ ]:
facilities_df['ProposedStaff'] = facilities_df['StaffCount']
facilities_df

In [ ]:
min_staff_indices = facilities_df.index[facilities_df['ProposedStaff'] <  min_staff_in_area_facility_df['MinStaff']]
min_staff_indices

In [ ]:
print(min_staff_indices.values)

In [ ]:
for item in min_staff_indices:
    staff_count = facilities_df['ProposedStaff'][item]
    facilities_df['ProposedStaff'][item] = min_staff_in_area_facility_df['MinStaff'][item]
    near_facility = facilities_df['NearestAreaZipcode'][item]
    index = facilities_df[facilities_df['AreaZipcode'] == near_facility].index[0]
    facilities_df['ProposedStaff'][index] = facilities_df['ProposedStaff'][index] + staff_count - min_staff_in_area_facility_df['MinStaff'][item]
    print("Add " + str(min_staff_in_area_facility_df['MinStaff'][item] - staff_count) + " staff from " + facilities_df['FacilityID'][index] + " to " + facilities_df['FacilityID'][item])

In [ ]:
facilities_df

In [ ]:
proposed_staff_df = facilities_df[['FacilityID', 'AreaZipcode', 'StaffCount', 'ProposedStaff']]
proposed_staff_df

In [ ]:
proposed_staff_df.to_csv("proposed_staff.csv")